In [18]:
from ast import Interactive
from sympy import Q
from builtins import list
from locale import normalize
from re import S, X
from cProfile import label, run
import pickle
from bs4 import ResultSet
import numpy as np
from matplotlib import pyplot as plt
from env import *
from tqdm import tqdm
from qadapt import *

In [19]:
def read_from_file(filename):
    f = open(filename, "r")
    adj = list()
    lines = f.readlines()
    for line in lines:
        line = line[1:-2]
        line = line.split(", ")
        temp = list()
        for item in line:
            item = float(item)
            temp.append(item)
        adj.append(temp)
    f.close()

    return adj

adj = read_from_file("/home/rm2022/topo.txt")

In [20]:
network = Network(adj, group_num=33, drop=True)

duration = 10000
slot = 1
loads = np.arange(20, 100, 2.5)

## Training

In [21]:
agent = Qadapt(network, pre=True)
network.bind(agent)
network.reset()

result = network.train(duration=10000, slot=1, freq=1, lambd=40, inject=True, adv=4)
network.agent.store(f"/home/rm2022/DQN_routing/DRL/qadaptlargeadv4.pkl")

agent = Qadapt(network)
network.bind(agent)
network.reset()

network.agent.load(f"/home/rm2022/DQN_routing/DRL/qadaptlargeadv4.pkl")
result = network.train(duration=100000, slot=1, freq=1, lambd=40, inject=True, adv=4)
network.agent.store(f"/home/rm2022/DQN_routing/DRL/qadaptlargeadv4.pkl")

agent = Qadapt(network, static=True)
network.bind(agent)
network.reset()

network.agent.load(f"/home/rm2022/DQN_routing/DRL/qadaptlargeadv4.pkl")
for j in list(network.agent.QTable[2].items()):
    print(j)
result = network.train(duration=10000, slot=1, freq=1, lambd=40, inject=True, adv=4)
plt.plot(result['route_time'])
plt.savefig(f"adapted.png")
plt.close()
print(f"completed with {result['route_time'][-1]}")

100%|██████████| 100000/100000 [06:50<00:00, 243.61it/s]


(0, {1: 8.663119939853878, 2: 9.058044288328446, 3: 8.897689206990709, 4: 8.897689206990709, 5: 8.510623853050785, 6: 9.139438675079239, 7: 1.4735376630326198, 239: 10, 247: 10, 255: 10, 263: 10})
(1, {0: 10, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10, 7: 8.663119939853878, 206: 10, 214: 10, 222: 10, 230: 10})
(2, {0: 10, 1: 10, 3: 10, 4: 10, 5: 10, 6: 10, 7: 9.058044288328446, 173: 10, 181: 10, 189: 10, 197: 10})
(3, {0: 10, 1: 10, 2: 10, 4: 10, 5: 10, 6: 10, 7: 8.897689206990709, 140: 10, 148: 10, 156: 10, 164: 10})
(4, {0: 10, 1: 10, 2: 10, 3: 10, 5: 10, 6: 10, 7: 8.897689206990709, 107: 10, 115: 10, 123: 10, 131: 10})
(5, {0: 10, 1: 10, 2: 10, 3: 10, 4: 10, 6: 10, 7: 8.510623853050785, 74: 10, 82: 10, 90: 10, 98: 10})
(6, {0: 10, 1: 10, 2: 10, 3: 10, 4: 10, 5: 10, 7: 9.139438675079239, 41: 10, 49: 10, 57: 10, 65: 10})
(7, {0: 10, 1: 10, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10, 8: 10, 16: 1.2097999058305768, 24: 10, 32: 10})
(8, {9: 8.818712314920802, 10: 9.221655227352766, 11: 9.221655227352766, 

100%|██████████| 10000/10000 [00:40<00:00, 249.59it/s]

completed with 976.9511762370341


## Testing

In [ ]:
agent = Qadapt(network, static=True)
network.bind(agent)
network.reset()

f = open(f"/home/rm2022/DQN_routing/DRL/qadapt.txt", "a")
qadapt_res = list()
for load in loads:
    for i in range(1):
        run_res = list()
        network.reset()
        network.agent.load(f"/home/rm2022/DQN_routing/DRL/qadaptlargeadv4.pkl")
        result = network.train(duration=10000, slot=1, freq=1, lambd=load, inject=True, adv=4)
        run_res.append(result['route_time'][-1])
    qadapt_res.append(np.min(run_res))
    print(f"{load} completed with {np.min(run_res)}")
    
f.write(repr(qadapt_res))
f.write("\n")
f.close()